In [ ]:
!pip install pymongo

In [28]:
import pprint
import re
import requests
import tarfile
from datetime import datetime
from glob import glob
from pymongo import MongoClient
import key_param

In [29]:
url = 'https://www.rondhuit.com/download/ldcc-20140209.tar.gz'
target_path = 'ldcc-20140209.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())


file = tarfile.open('ldcc-20140209.tar.gz')
file.extractall()
file.close

<bound method TarFile.close of <tarfile.TarFile object at 0x107ff9f00>>

In [30]:
topic_news_list = glob('./text/topic-news/[!LICENSE]*')
data = []
for i, news_file in enumerate(topic_news_list):
    data.append({'url':'', 'datetime_of_topic': '', 'contents': ''})
    data[i]['url']=''
    data[i]['datetime_of_topic'] = ''
    data[i]['title'] = ''
    data[i]['contents'] = ''
    data[i]['relative_articles'] = []
    data[i]['relative_info'] = []
    k=0
    with open(topic_news_list[i]) as f:
        for j, line in enumerate(f):
            if j==0:
                data[i]['url']=line.strip('\n')
            elif j==1:
                data[i]['datetime_of_topic'] = datetime.strptime(line.strip('\n'), '%Y-%m-%dT%H:%M:%S%z')
            elif j==2:
                data[i]['title'] = line.strip('\n')
            else:
                if re.search("関連記事", line):
                    k=1
                elif re.search("関連情報", line):
                    k=2
                else:
                    if k == 0:
                        data[i]['contents'] = data[i]['contents'] + line
                    elif k == 1:
                        if line.strip('\n') != '':
                            data[i]['relative_articles'].append(line.strip('\n'))
                    elif k == 2:
                        if line.strip('\n') != '':
                            data[i]['relative_info'].append(line.strip('\n'))
                
                



In [31]:
client = MongoClient(key_param.MONGO_URI)
db = client['livedoor_news']
collection = db['topic_news']

if collection.count_documents({}) != 0:
    collection.delete_many({})

result = collection.insert_many(data)
print(f'{len(result.inserted_ids)} documents are inserted')

770 documents are inserted


In [42]:
pipeline = [
  {
    '$search': {
      'index': "topic_news_1",
      'text': {
        'query': "ダウンタウン",
        'path': 'contents'
      }
    }
  }
]

In [43]:
pprint.pprint(list(collection.aggregate(pipeline))[0])

{'_id': ObjectId('65d30e336d6d4ba77a0301f9'),
 'contents': '20日に放送されたダウンタウンDXに、佐藤かよと幼馴染のモデル菜々緒が同時に出演した。\n'
             '\n'
             '実際に、子供の頃の写真が何枚も公開されたので、2人が昔からの知り合いだったことは間違いないだろう。\n'
             '\n'
             '菜々緒が佐藤かよと「13年ぶりに再会したら、女の子になっていた」というのだから、“衝撃の再会”である。\n'
             '\n'
             '13年の間に女性へ変身した佐藤かよの美貌は、アンジャッシュの渡部が、本気で口説いてしまったほど。\n'
             '\n'
             '番組内で、はるな愛が「男の部分を出せばいい」とアドバイスしていたが、佐藤かよが男らしさを見せる日は来るのだろうか。\n'
             '\n',
 'datetime_of_topic': datetime.datetime(2011, 10, 21, 10, 21),
 'relative_articles': ['人気モデル菜々緒、佐藤かよを本気で口説いた芸人とは？',
                       '・http://news.livedoor.com/article/detail/5275940/',
                       'アンジャッシュ渡部の発言に「最低！」の声が続出',
                       '・http://news.livedoor.com/article/detail/5934334/'],
 'relative_info': [],
 'title': '佐藤かよの意外な幼馴染とは?',
 'url': 'http://news.livedoor.com/article/detail/5956985/'}


In [ ]:
pipeline = [
  {
    '$search': {
      'index': "topic_news_1",
      'text': {
        'query': "ダウンタウン",
        'path': {
          'value': 'contents', 'multi': 'ngram'
        }
      }
    }
  }
]

In [39]:
pprint.pprint(list(collection.aggregate(pipeline))[0])

{'_id': ObjectId('65d30e336d6d4ba77a0302a7'),
 'contents': 'ダウンタウンのガキの使いやあらへんで！！”10月9日放送回で、ダウンタウンがマスターとママになり、客である吉本の芸人同士で酒を飲ませて話をさせる“べろんべろん酒場”という企画があった。\n'
             '\n'
             'ゲストの一人のココリコ遠藤が“入店”してからしばらく突然立ち上がり、「マスターにご挨拶していなかったので」とダウンタウン浜田に挨拶をした。\n'
             '\n'
             'そこから、遠藤が吉本の若手の挨拶がなっていないと語り始める。\n'
             '\n'
             '同じくゲストのフットボール岩尾に対し「お前らより下の世代におるやろ。20歳そこそこの。違う番組、違う現場でも、同じ局に入れば挨拶に（行くべき）」と主張したのだ。\n'
             '\n'
             'ここで思い出されるのは、芸能界を引退した島田紳助さんも、2009年のオールスター感謝祭で、若手芸人を恫喝した要因に「挨拶が出来ていない」というものがあった。\n'
             '\n'
             '・「お前ら、潰すぞ！」生放送中に東京03を恫喝した島田紳助\u3000その後の和解劇真相\n'
             '\n'
             'しかし、挨拶を重視した紳助さんを彷彿させるようなココリコ遠藤の“年功序列”重視を批判する芸人が現れる。\n'
             '\n'
             '山崎邦正だ。「吉本の“タテ”がおかしくなってる」と説明され、「挨拶はしなきゃあかんわ」と山崎も最初は笑顔で同意していたが、その後、遠藤の方に目を向けつつ「芸人は年功序列じゃないからな」と言い放った。\n'
             '\n'
             'その瞬間から、山崎はどんどんとヒートアップしていく。「芸人は年功序列関係ない\u3000'
             '面白さ」「俺はやったるで！」。ここで笑い声がスタッフ側か、どこからか沸くも、それを聞いた山崎邦正は

In [44]:
pipeline = [
  {
    '$search': {
      'index': "topic_news_1",
      'text': {
        'query': "ダウンチウン",
        'path': 'contents',
        'fuzzy': {'maxEdits':2}
      }
    }
  }
]
pprint.pprint(list(collection.aggregate(pipeline))[0])

{'_id': ObjectId('65d30e336d6d4ba77a0302dc'),
 'contents': '20日放送「ダウンタウンDX」でえなりかずきが意外な趣味を告白した。\n'
             '\n'
             'えなりかずきと言えば、若者らしからぬ品行方正な佇まいで知られるが、そのイメージを自らブチ壊してしまったのだ。\n'
             '\n'
             'えなりかずきの趣味はなんと、「エヴァンゲリオン」のアテレコ。赤ワインを用意し、だらしなくソファに寝そべりながら、「逃げちゃダメだ、逃げちゃダメだ・・・」と、ノリノリで名場面を熱演。セリフだけでなく、動きもピッタリ合っているほど完璧だった。\n'
             '\n'
             'えなりかずきの意外な一面に、Twitterでは「キモい」「親近感が沸いた」と賛否両論。\n'
             '\n'
             '心なしか、同席していた泉ピン子は、えなりかずきの若者らしい私生活に顔をしかめているようだった。\n',
 'datetime_of_topic': datetime.datetime(2011, 10, 21, 3, 48),
 'relative_articles': [],
 'relative_info': [],
 'title': 'えなりかずきの私生活に「キモい」',
 'url': 'http://news.livedoor.com/article/detail/5955817/'}


In [47]:
pipeline = [
  {
    '$search': {
      'index': "topic_news_1",
      'text': {
        'query': "ダウンチウン",
        'path': 'contents',
        'fuzzy': {'maxEdits':2}
      }
    }
  },
  {
    '$project': {
      'title':1,
      'contents': 1,
      'datetime_of_topic':1,
      'score': {'$meta': 'searchScore'}
    }
  }
]
pprint.pprint(list(collection.aggregate(pipeline))[0])

{'_id': ObjectId('65d30e336d6d4ba77a0302dc'),
 'contents': '20日放送「ダウンタウンDX」でえなりかずきが意外な趣味を告白した。\n'
             '\n'
             'えなりかずきと言えば、若者らしからぬ品行方正な佇まいで知られるが、そのイメージを自らブチ壊してしまったのだ。\n'
             '\n'
             'えなりかずきの趣味はなんと、「エヴァンゲリオン」のアテレコ。赤ワインを用意し、だらしなくソファに寝そべりながら、「逃げちゃダメだ、逃げちゃダメだ・・・」と、ノリノリで名場面を熱演。セリフだけでなく、動きもピッタリ合っているほど完璧だった。\n'
             '\n'
             'えなりかずきの意外な一面に、Twitterでは「キモい」「親近感が沸いた」と賛否両論。\n'
             '\n'
             '心なしか、同席していた泉ピン子は、えなりかずきの若者らしい私生活に顔をしかめているようだった。\n',
 'datetime_of_topic': datetime.datetime(2011, 10, 21, 3, 48),
 'score': 2.504638195037842,
 'title': 'えなりかずきの私生活に「キモい」'}
